In [1]:
## Binary Customer Churn

# A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

# The data is saved as customer_churn.csv. Here are the fields and their definitions:

#     Name : Name of the latest contact at Company
#     Age: Customer Age
#     Total_Purchase: Total Ads Purchased
#     Account_Manager: Binary 0=No manager, 1= Account manager assigned
#     Years: Totaly Years as a customer
#     Num_sites: Number of websites that use the service.
#     Onboard_date: Date that the name of the latest contact was onboarded
#     Location: Client HQ Address
#     Company: Name of Client Company
    
# Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logRegression').getOrCreate()

In [3]:
data = spark.read.csv('/FileStore/tables/customer_churn.csv', inferSchema=True, header=True)

In [4]:
data.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: timestamp (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- Churn: integer (nullable = true)

In [5]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
summary| Names| Age| Total_Purchase| Account_Manager| Years| Num_Sites| Location| Company| Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
 count| 900| 900| 900| 900| 900| 900| 900| 900| 900|
 mean| null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777| null| null|0.16666666666666666|
 stddev| null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| null| null| 0.3728852122772358|
 min| Aaron King| 22.0| 100.0| 0| 1.0| 3.0|00103 Jeffrey Cre...| Abbott-Thompson| 0|
 max|Zachary Walsh| 65.0| 18026.01| 1| 9.15| 14.0|Unit 9800 Box 287...|Zuniga, Clark and...| 1|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+

In [6]:
data.columns

Out[ 6 ]: 
['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [7]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [8]:
output = assembler.transform(data)

In [9]:
final_data = output.select('features', 'churn')

In [10]:
train_churn, test_churn = final_data.randomSplit([0.7,0.3])

In [11]:
from pyspark.ml.classification import LogisticRegression

In [12]:
lr_churn = LogisticRegression(labelCol='churn')

In [13]:
fitted_churn_model = lr_churn.fit(train_churn)

In [14]:
training_sum = fitted_churn_model.summary

In [15]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
summary| churn| prediction|
+-------+-------------------+-------------------+
 count| 637| 637|
 mean| 0.1695447409733124|0.13814756671899528|
 stddev|0.37552723518144804|0.34532596516187675|
 min| 0.0| 0.0|
 max| 1.0| 1.0|
+-------+-------------------+-------------------+

In [16]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
pred_and_labels = fitted_churn_model.evaluate(test_churn)
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|churn| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
[25.0,9672.03,0.0...| 0|[5.33697915578766...|[0.99521264878647...| 0.0|
[26.0,8787.39,1.0...| 1|[0.20974272545996...|[0.55224429401690...| 0.0|
[26.0,8939.61,0.0...| 0|[7.34407304052111...|[0.99935400529058...| 0.0|
[28.0,11204.23,0....| 0|[1.78794975731964...|[0.85667572678924...| 0.0|
[29.0,5900.78,1.0...| 0|[4.41708532099747...|[0.98807457288940...| 0.0|
[31.0,5304.6,0.0,...| 0|[3.66122893213312...|[0.97494307704928...| 0.0|
[31.0,5387.75,0.0...| 0|[2.71976591887789...|[0.93818295943165...| 0.0|
[31.0,9574.89,0.0...| 0|[3.81975321588822...|[0.97853752797540...| 0.0|
[31.0,10182.6,1.0...| 0|[5.30967986991282...|[0.99508080903458...| 0.0|
[32.0,9036.27,0.0...| 0|[-0.5543889934935...|[0.36484672971466...| 1.0|
[32.0,9472.72,1.0...| 0|[3.98726762997487...|[0.98178751630139...| 0.0|
[32.0,11540.86,0....| 0|[7.85198748358165...|[0.99961117313325...| 0.0|
[33.0,7492.9,0.0,...| 0|[5.58874264588769...|[0.99627420638026...| 0.0|
[33.0,7750.54,1.0...| 0|[4.73141410680167...|[0.99126300953106...| 0.0|
[33.0,8556.73,0.0...| 0|[4.13730483529762...|[0.98428507759189...| 0.0|
[33.0,10306.21,1....| 0|[2.11669570908128...|[0.89251535416058...| 0.0|
[33.0,10309.71,1....| 0|[7.15228676064832...|[0.99921754138532...| 0.0|
[33.0,11370.28,1....| 0|[7.73587763996816...|[0.99956332197239...| 0.0|
[33.0,12115.91,1....| 0|[2.61197786303336...|[0.93162848752142...| 0.0|
[33.0,12249.96,0....| 0|[6.31979781671382...|[0.99820292782093...| 0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [17]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')
auc = churn_eval.evaluate(pred_and_labels.predictions)
auc

Out[ 20 ]: 0.7421353156647275

In [18]:
final_lr_model = lr_churn.fit(final_data)
new_customers = spark.read.csv('/FileStore/tables/new_customers.csv',inferSchema=True,
                              header=True)
new_customers.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: timestamp (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)

In [19]:
test_new_customers = assembler.transform(new_customers)
test_new_customers.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: timestamp (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- features: vector (nullable = true)

In [20]:
final_results = final_lr_model.transform(test_new_customers)
final_results.select('Company','prediction').show()

+----------------+----------+
 Company|prediction|
+----------------+----------+
 King Ltd| 0.0|
 Cannon-Benson| 1.0|
Barron-Robertson| 1.0|
 Sexton-Golden| 1.0|
 Wood LLC| 0.0|
 Parks-Robbins| 1.0|
+----------------+----------+